<a href="https://colab.research.google.com/github/kingsman0909/MyPortfolio/blob/main/bark_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# 🐍 Install Tortoise TTS and dependencies
!pip install -q git+https://github.com/camenduru/tortoise-tts.git
!pip install -q gradio einops transformers scipy

# 🛠 Import required modules
import os
import torch
import torchaudio
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_voice, load_voices

# 🔧 Set up model
tts = TextToSpeech()

# 🎤 Choose voice (you can also use custom voice cloning later)
VOICE = 'pat'  # 👈 Try: 'pat', 'william', 'daniel', 'deniro', 'train_dreams', 'lj'
TEXT = "My body may fall—but my will won't."

# 📦 Generate voice
gen = tts.tts(
    text=TEXT,
    voice_samples=load_voice(VOICE),
    conditioning_latents=None,
    preset='high_quality',  # Options: 'ultra_fast', 'fast', 'standard', 'high_quality'
)

# 💾 Save output
OUTPUT_PATH = "Thorfin_Tortoise.wav"
torchaudio.save(OUTPUT_PATH, gen.squeeze(0).cpu(), 24000)
print(f"✅ Saved to {OUTPUT_PATH}")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b1f245|OK  |       0B/s|/root/.cache/suno/bark_v0/6285677e88715abde42a9924db939b3b.pt

Status Legend:
(OK):download completed.
✅ Available voice prompts:
 - Unconditional
 - Announcer
 - Speaker 0 (en)
 - Speaker 1 (en)
 - Speaker 2 (en)
 - Speaker 3 (en)
 - Speaker 4 (en)
 - Speaker 5 (en)
 - Speaker 6 (en)
 - Speaker 7 (en)
 - Speaker 8 (en)
 - Speaker 9 (en)
 - Speaker 0 (de)
 - Speaker 1 (de)
 - Speaker 2 (de)
 - Speaker 3 (de)
 - Speaker 4 (de)
 - Speaker 5 (de)
 - Speaker 6 (de)
 - Speaker 7 (de)
 - Speaker 8 (de)
 - Speaker 9 (de)
 - Speaker 0 (es)
 - Speaker 1 (es)
 - Speaker 2 (es)
 - Speaker 3 (es)
 - Sp

100%|██████████| 21/21 [00:28<00:00,  1.34s/it]


✅ Audio saved to Thorfin.wav
